In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic, FlCollection
from python.psfs import PsfVectorial
from python.estimators import est_quad2Diter
from python.estimators import est_quadraticdirect1D
from python.simulators import Simulator

In [ ]:
psf_vecpp = PsfVectorial()
psf_vecpp.setpinhole(AU=1)
phaseplateposmm = np.arange(6)/10  # mm
zpos = [-200, -100, 0, 100, 200,]
phaseplateposrel = phaseplateposmm/2.5  # pupil diameter assumed to be 5 mm; 
stdx = np.zeros((len(phaseplateposrel),len(zpos),3))
crb1 = np.zeros((len(phaseplateposrel),len(zpos),3))
biasx = np.zeros((len(phaseplateposrel),len(zpos),3))
rmsex = np.zeros((len(phaseplateposrel),len(zpos),3))
phot = np.zeros((len(phaseplateposrel),len(zpos),1))

In [ ]:
fl = FlStatic(brightness=1000)  # define a static fluorophore
fl.pos = [10, 0, 0]

sim = Simulator(fluorophores=fl)

numberOfLocalizations=1000

# define scan pattern
L = 75
zeroposx = np.atleast_2d(np.array([-1,1,0])*L/2)
probecenter = True  # should we also probe the center?
orbitpoints = 6
laserpower = 10  # relative, increases brightness
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 1  # how often to repeat the pattern scan

In [ ]:
psfall = []
stdx = np.zeros((len(phaseplateposrel), len(zpos), 3))
crb1 = np.zeros((len(phaseplateposrel), len(zpos), 3))
biasx = np.zeros((len(phaseplateposrel), len(zpos), 3))
rmsex = np.zeros((len(phaseplateposrel), len(zpos), 3))
phot = np.zeros((len(phaseplateposrel), len(zpos)))
for k in range(len(phaseplateposrel)):
    sys_mis = {}
    sys_mis['maskshift'] = [phaseplateposrel[k], 0]  # radius of pupil function is 1
    psf_vecpp.setpar(**sys_mis)
    sim.definePattern("donut_misaligned", psf_vecpp, 
                      phasemask="vortex", 
                      makepattern="orbitscan", 
                      orbitpoints=orbitpoints, 
                      probecenter=probecenter,
                      orbitL=L,
                      pointdwelltime=pointdwelltime,
                      laserpower=laserpower,
                      repetitions=repetitions)
    stack, gridv = psf_vecpp.imagestack("vortex")
    psfall.append(stack)
    for z in range(len(zpos)):
        sim.fluorophores.pos = [0, 0, zpos[z]]
        seq = ["donut_misaligned", "estdonut"]
        out = sim.runSequence(seq)
        sr = sim.summarize_results(out)
        stdx[k,z,:] = sr.std
        crb1[k,z,:] = sr.sCRB1
        biasx[k,z,:] = sr.bias
        rmsex[k,z,:] = sr.rmse
        phot[k,z] = sr.phot

stdxrel=stdx/crb1[0,:,:]*np.sqrt(phot)  # normalized to perfectly aligned phaseplate and photon numbers
rmserel=rmsex/crb1[0,:,:]*np.sqrt(phot)  # normalized to perfectly aligned phaseplate and photon numbers